# 1. Extracting data from *Common Fragrance and Flavor Materials*

## Purpose, contents, & conclusions

**Purpose:** This notebook was used to extract data about aroma/flavor chemicals from a PDF copy of *Common Fragrance and Flavor Materials*. Data is given only basic cleaning and treatment. *Note:* The PDF copy is excluded from the public repository for copyright reasons.

**Reference:** Surburg, H., & Panten, J. (Eds.). (2016). Common Fragrance and Flavor Materials. Wiley. https://doi.org/10.1002/9783527693153

**Contents:** The notebook contains:
* Extraction of data from one ~300-page PDF.

**Conclusions:** Key outputs are:
* Lightly cleaned data are written to the `common-materials` table of the `aromas-flavors.db` SQLite database.

## Test drive `pdfminer` again

The PDF contains 326 pages. From visual inspection, the chemical information of interest **begins on page 20 and ends on page 249**.

In [48]:
pagerange = [19, 25]

In [39]:
from pdfminer.high_level import extract_text

#### Via `extract_text`

In [41]:
extracted = extract_text("raw-data/common-fragrance-and-flavor-materials.pdf",
                         page_numbers = [page for page in range(pagerange[0], pagerange[1])]
                        )

In [43]:
extracted[:2000]

'8\n\nIndividual Fragrance and Flavor Materials\n\nSymrise\nTakasago\nTreatt\nVioryl\n\n= Symrise GmbH & Co KG, Germany\n= Takasago Perfumery Co., Japan\n= R.C. Treatt & Co., Ltd., UK\n= Vioryl S.A., Greece\n\nMonographs on fragrance materials and essential oils which have been published by\nthe Research Institute for Fragrance Materials (RIFM) in “Food and Chemical Toxi-\ncology” are cited below the individual compounds as “FCT” with year, volume, and\npage of publication.\n\n2.1 Aliphatic Compounds\n\nThe acyclic terpenes are discussed separately in Section 2.2. Some of the cycloali-\nphatic fragrance and flavor materials are structurally related to the cyclic terpenes\nand are, therefore, discussed in Section 2.4 after the cyclic terpenes.\n\n2.1.1 Hydrocarbons\n\nSaturated and unsaturated aliphatic hydrocarbons with straight as well as branched\nchains occur abundantly in natural foodstuffs, but they contribute to the odor and\ntaste only to a limited extent and have not therefore 

This output is pretty reasonable. It has a fair number of extra line breaks, but that's fixable. 

*Common Fragrance and Flavor Chemicals* is more simply formatted than *Fenaroli's Handbook* was, so less of the content seems to be in a jumbled order. In fact, it might be possible to fully parse out the chemical data via regex using the unique CAS numbers as punctuation because it looks like they are always enclosed in square brackets, e.g.: "[51447-08-6]" with trailing line breaks.

In [77]:
import re

In [ ]:
regex = r""
    found = re.search(regex, chem_desc)

#### Via `extract_pages`

In [51]:
from pdfminer.high_level import extract_pages

In [52]:
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar

In [53]:
extracted = []
for page in extract_pages("raw-data/common-fragrance-and-flavor-materials.pdf",
                          page_numbers = [page for page in range(pagerange[0], pagerange[1])]):
    for element in page:
        extracted.append(element)

Apparently reading the text by character rather than text box is also an option. Let's try that.

In [55]:
extracted_chars = []
for page in extract_pages("raw-data/common-fragrance-and-flavor-materials.pdf",
                          page_numbers = [page for page in range(pagerange[0], pagerange[1])]
                          ):
    for element in page:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                for character in text_line:
                    if isinstance(character, LTChar):
                        extracted_chars.append((character._text, character.fontname, round(character.size, 1)))

In [76]:
extracted_chars[:10]

[('8', 'BGZQZR+AdvPS_TIR', 10.4),
 ('I', 'DVTDJU+AdvPS_TII', 8.5),
 ('n', 'DVTDJU+AdvPS_TII', 8.5),
 ('d', 'DVTDJU+AdvPS_TII', 8.5),
 ('i', 'DVTDJU+AdvPS_TII', 8.5),
 ('v', 'DVTDJU+AdvPS_TII', 8.5),
 ('i', 'DVTDJU+AdvPS_TII', 8.5),
 ('d', 'DVTDJU+AdvPS_TII', 8.5),
 ('u', 'DVTDJU+AdvPS_TII', 8.5),
 ('a', 'DVTDJU+AdvPS_TII', 8.5)]

Is this text separable by font size like *Fenaroli's Handbook* was?

In [60]:
fontsizes = []
for char in extracted_chars:
    fontsizes.append(char[2])
print("Font sizes found: ", set(fontsizes))

Font sizes found:  {7.3, 8.5, 9.1, 10.4, 12.8, 17.1}


In [67]:
sep_by_size = {}
for char in extracted_chars:
    try:
        sep_by_size[char[2]] += char[0]
    except KeyError:
        sep_by_size[char[2]] = ""

In [73]:
for key in sep_by_size:
    print("Font size: ", key)
    print(sep_by_size[key][:1000])
    print("====")

Font size:  10.4
MonographsonfragrancematerialsandessentialoilswhichhavebeenpublishedbytheResearchInstituteforFragranceMaterials(RIFM)in“FoodandChemicalToxi-cology”arecitedbelowtheindividualcompoundsas“FCT”withyear,volume,andpageofpublication.TheacyclicterpenesarediscussedseparatelyinSection2.2.Someofthecycloali-phaticfragranceandflavormaterialsarestructurallyrelatedtothecyclicterpenesandare,therefore,discussedinSection2.4afterthecyclicterpenes.Saturatedandunsaturatedaliphatichydrocarbonswithstraightaswellasbranchedchainsoccurabundantlyinnaturalfoodstuffs,buttheycontributetotheodorandtasteonlytoalimitedextentandhavenotthereforegainedcommercialimportance.Theonlyexceptionsarethehighlyunsaturatedhydrocarbons(E,Z)-1,3,5-undeca-trieneanditssyntheticprecursor1,3-undecadien-5-yne.(E,Z)-1,3,5-Undecatriene[51447-08-6]CH,M150.26,isacolorlessliquidwithastronggreen,galbanum-likeodor.Itoccursnaturallyingalbanumoil(seep.207)andistheodor-determiningconstituentoftheoil.Thecommercialqualitiesalsocontai

This could be a convenient break! The header information (here, the first "A") is in a unique font size. Each chemical entry (here, "ACACIA GUM") is both in all-caps and a unique font size. Finally, within that, data labels (such as "Botanical name:") are bolded. This font information could be a useful way to parse the document.

#### Try parsing the document by font formatting.

I'll do this in some nice, organized functions so that after I troubleshoot them, I can easily reuse them to extract the entire ~2000 page document.

In [17]:
def read_line(textline: str) -> str:
    
    """
    Called:
         By parse_pages function for partial parsing of a PDF document.
    Accepts:
         textline: A LTTextLine object from partial PDF parsing by pdfminer
    Returns:
         a string of text from the TextLine object, terminating with a pipe (|)
    """
    
    text = []
    for character in textline:
        if isinstance(character, LTChar):
            text.append(character._text)
    text.append("|") # Include pipe as a separator at the end of each text container.
                     # This will be a fallback in case values cannot readily be parsed
                     # a more elegant way down the road.
    return "".join(text)

In [18]:
def start_new_entry_content(segmented_entry: list) -> str:
    
    """
    Called:
        By parse_pages function whenever a new encyclopedia entry is detected.
    Purpose:
        Clean up the *prior entry's* content, which at the time of calling is
        stored as a disjointed list of strings, segmented_entry.
    Accepts:
        segmented_entry: A list of strings containing the body of reference
              information for a single encyclopedia entry. The strings are
              of varied length, and their length isn't particularly
              meaningful.
    Returns:
        a single string combining all the entries from the input list.
    """
    
    return " ".join(segmented_entry)

In [19]:
# Display a progress bar so this long process doesn't occur
# entirely in a black box.
from tqdm import tqdm

In [20]:
def parse_pages(start_page: int, end_page: int) -> tuple:
    
    """
    Called:
        By user.
    Purpose:
        Manage data extraction from the multipage Fenaroli's Handbook PDF.
    Accepts:
        start_page: int indicating the first page of the PDF for extraction
        end_page: int indicating the final page of the PDF for extraction
    Returns:
        a tuple containing:
            - a list of titles for articles from the encyclopedia-style book
            - a list containing the corresponding article content (with one
              list entry for each article)
    """
    
    
    extracted = []
    entry_titles = []
    entry_content = []
    partial_content = []

    for page in tqdm(extract_pages("raw-data/fenarolis-handbook.pdf",
                                   page_numbers = [page for page in range(start_page, end_page)])):
        for element in page:
            # extracted.append(element)
            if isinstance(element, LTTextContainer):
                for text in element:
                    
                    if isinstance(text, LTTextLine):
                        # If we are, in fact, iterating over a TextLine object
                        # as expected instead of, e.g., a LTChar single
                        # character, then proceed. Single characters in text
                        # containers will be ignored.
                        
                        # First, extract the text from this line.
                        newline = read_line(text)

                        # Then figure out how to categorize the line.
                        # Determine the font size for this line based on the first
                        # character present. Is this the title of an encyclopedia entry
                        # (font size 11), the body of an entry (font size 9.5), or
                        # something else (in which case it can be ignored)?
                        chars = [char for char in text]
                        if isinstance(chars[0], LTChar):
                            if round(chars[0].size, 1) == 11:
                                entry_titles.append(newline)
                                # When a new entry is detected, wrap up the old entry &
                                # prepare to start a fresh new one.
                                entry_content.append(start_new_entry_content(partial_content))
                                partial_content = []
                            elif round(chars[0].size, 1) == 9.5:
                                partial_content.append(newline)
                            else:
                                pass
    
    # Before finishing, append the final partial_content to entry_content, if any
    entry_content.append(start_new_entry_content(partial_content))
      
    return entry_titles, entry_content

Now, let's test these functions to make sure everything is working as expected:

In [21]:
entry_titles, entry_content = parse_pages(25, 30)

5it [00:02,  1.69it/s]


In [22]:
entry_titles

['ACACIA GUM|',
 'ACETAL|',
 'ACETALDEHYDE|',
 'ACETALDEHYDE, BUTYL PHENETHYL ACETAL|',
 'ACETALDEHYDE DI-cis-3-HEXENYL ACETAL|']

In [23]:
entry_content

['',
 'Botanical name:.Acacia senegal.(L.).Willd.| Botanical family:.Leguminosae| Other names:.Acacia.[JAN];.Acacia senegal.gum;.Arabic.gum;.gum.Arabic;.Acacia delbata.gum;.Acacia.solution;.Acacia.syrup;.| Australian.gum;.Gum.Arabic;.Indian.gum;.Wattle.gum| CAS.No.:| CoE.No.:| 9000-01-05| n/a| FL.No.:| EINECS.No.:232-519-5| n/a| FEMA.No.:| JECFA.No.:| 2001| n/a| NAS.No.:| E.No.:| 2001| 414| Description:.Arabic.or.acacia.gum.is.the.dried.exudate.obtained.from.the.stems.and.branches.of.Acacia senegal (L.).Willd..or.of.| related.species.of.Acacia..Injured.trees.exude.gum.Arabic;.heat,.poor.nutrition.and.drought.stimulate.its.production..Most.of.the.gum.| Arabic.production.is.from.wild.trees,.but.some.from.privately.owned.and.cultivated.gardens.are.tapped.and.collected.on.a.systematic.| basis.| The.gum.called.Hashab.geneina.(garden.gum).is.the.cleanest.and.lightest.grade.and.is.most.preferred.for.the.U.S..market..The.wild.| gum.(called.Hashab.wady).is.collected.on.a.part-time.basis.in.the.

**Fantastic.** This is working fairly nicely. The last thing that needs to be done (bare minimum) before turning this loose on the full document is to extract out the unique CAS identification numbers. These will be the most reliable way for searching the data later.

#### Extract unique CAS numbers

Because of the structure of the PDF document & my approach to extracting it, the unique CAS numbers that we need have an extraneous data label ("CoE.No.") between the "CAS No." label and the number itself, e.g.: "CAS.No.:| CoE.No.:| 105-57-7|" So, the simplest way to find the CAS number will likely be to search for the extraneous "CoE.No." and snag the number that follows it.

In [24]:
import re

In [25]:
def extract_cas_num(chem_desc: str) -> str:
        
    """
    Called:
        By user.
    Purpose:
        Extract a chemical's unique CAS number identifier from the chemical
        description extracted from the Fenaroli's Handbook.
    Accepts:
        chem_desc: String containing the entirety of one chemical's entry
            in the Fenaroli's Handbook.
    Returns:
        a string: the unique CAS number
    """
    
    regex = r"CoE.No.:\|\s[\d\-]+"
    found = re.search(regex, chem_desc)
    if found:
        cas_num = re.sub("CoE.No.:\|\s", "", found.group(0))
        return cas_num
    else:
        pass

In [26]:
extract_cas_num(entry_content[2])

'105-57-7'

## Extract the complete data set

Using the functions above, extract the complete data set from the Fenaroli's Handbook. At this stage, we will generate a fairly crude data set:
* `titles`: A list of article titles (each corresponding to an individual chemical name) for each entry in the Handbook.
* `descriptions`: A list containing the complete full text of the Handbook, with one list entry per chemical.
* `cas_nums`: A list of unique CAS numbers, with one list entry per chemical.

### Parse the PDF

Using the functions above, crudely parse the PDF into lists of chemical `titles` and their `descriptions.

In [27]:
titles, descriptions = parse_pages(25, 2061)

2036it [06:59,  4.86it/s]


Double-check that an equal number of entry titles and entry descriptions were found:

Remember that the list of extracted extracted data entries, `descriptions`, has an unnecessary blank entry at the beginning (because of line 51 in `parse_pages`). Remove that before proceeding.

In [28]:
descriptions.pop(0)

''

In [29]:
len(titles), len(descriptions)

(2735, 2735)

### Extract CAS numbers

Generate a list of CAS numbers, `cas_nums`, to serve as unique identifiers for each chemical entry. The CAS numbers are currently buried in the list of `descriptions`.

In [30]:
cas_nums = []
for desc in descriptions:
    cas_nums.append(extract_cas_num(desc))

### Clean up the descriptions

The `descriptions` are pretty ugly right now, featuring a lot of unnecessary periods that are artifacts from the PDF reading, as well as a bunch of pipes (|) we inserted earlier as a backup plan for text separation. Go a head and remove both.

In [31]:
clean_descriptions = [desc.replace(".", " ").replace("|", "") for desc in descriptions]

### Fix the casing of entry `titles`

The list of `titles` are currently being yelled in all caps. Replace that with title case.

In [32]:
clean_titles = [title.replace("|", "").title() for title in titles] # Hey, I heard you like titles.

### Bundle the extracted data into a dict

Combine the lists of `titles`, `descriptions`, and `cas_nums` into a single list of dicts.

In [33]:
len(clean_titles), len(clean_descriptions), len(cas_nums)

(2735, 2735, 2735)

In [34]:
aromas = []
for i in range(len(clean_titles)):
    aromas.append(dict(CAS_num = cas_nums[i],
                       name = clean_titles[i],
                       full_description = clean_descriptions[i]))

In [35]:
aromas[0]

{'CAS_num': '9000-01-05',
 'name': 'Acacia Gum',
 'full_description': 'Botanical name: Acacia senegal (L ) Willd  Botanical family: Leguminosae Other names: Acacia [JAN]; Acacia senegal gum; Arabic gum; gum Arabic; Acacia delbata gum; Acacia solution; Acacia syrup;  Australian gum; Gum Arabic; Indian gum; Wattle gum CAS No : CoE No : 9000-01-05 n/a FL No : EINECS No :232-519-5 n/a FEMA No : JECFA No : 2001 n/a NAS No : E No : 2001 414 Description: Arabic or acacia gum is the dried exudate obtained from the stems and branches of Acacia senegal (L ) Willd  or of  related species of Acacia  Injured trees exude gum Arabic; heat, poor nutrition and drought stimulate its production  Most of the gum  Arabic production is from wild trees, but some from privately owned and cultivated gardens are tapped and collected on a systematic  basis  The gum called Hashab geneina (garden gum) is the cleanest and lightest grade and is most preferred for the U S  market  The wild  gum (called Hashab wady) i

## Write it to the database

We'll write the extracted data to the `fenaroli` table of the SQLite database, `aromas-flavors.db`.

In [36]:
import dataset

In [37]:
db = dataset.connect("sqlite:///aromas-flavors.db")

In [38]:
db["fenaroli"].insert_many(aromas)

Great -- all done. The entries from *Fenaroli's Handbook* have been extracted and written to `aromas-flavors.db`. They're searchable by chemical name and CAS number.